In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import BitsAndBytesConfig
import torch
import json
import bitsandbytes
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
# from unsloth import FastLanguageModel
# unslot doesn't work since unsloth need triton and triton isn't available on windows
import torch
from datasets import Dataset
import time

In [6]:
tokenizer = AutoTokenizer.from_pretrained("Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24")

In [7]:
# Create a BitsAndBytesConfig object with 8-bit loading
quant_config = BitsAndBytesConfig(load_in_4bit=True)

# Initialize an empty model with the correct configuration
with init_empty_weights():
    model = AutoModelForCausalLM.from_pretrained(
        "Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24",
        quantization_config=quant_config
    )

# Load the model with disk offloading
# model = load_checkpoint_and_dispatch(
#     model,
#     "Vikhrmodels/Vikhr-Llama3.1-8B-Instruct-R-21-09-24",
#     device_map="auto",
#     offload_folder="D:\mtrie\OneDrive\Downloads\MLClub\offload"  # Specify the folder for disk offloading
# )

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
# Example input_ids (replace with actual input data)
input_text = "Hello llama model"
input_ids = tokenizer.encode(input_text, return_tensors='pt')

# Ensure inputs are on the same device as the model
inputs = input_ids.to(next(model.parameters()).device)

# Generate outputs from the model
outputs = model.generate(inputs, max_new_tokens=100, use_cache=True)

# Batch decoding
text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

print(text)

Hello llama modelers! I am excited to share with you my latest creation, the Peruvian Long-necked Llama. This breed is known for its distinctive long neck and is a favorite among llama enthusiasts. So, let's dive into the details of this model and see how it can be used to enhance your knowledge of llama anatomy.
### Introduction to the Peruvian Long-necked Llama
The Peruvian Long-necked Llama is a breed that has been developed over the centuries for its unique characteristics.


In [4]:
import time  # Import the time module

text = ""  # Initialize text before the loop

while True:
    input_text = input("Enter input text: ")
    if input_text.lower() == 'exit':
        print("Goodbye!")
        break
    text = input_text
    print(f"Enter input text: {input_text}")
    
    batch = []

    # batch is one, we will be serving one person only. 
    for k in range(1):
        batch.append(text)

    # Tokenize input text
    input_ids = tokenizer.encode(input_text, return_tensors='pt')

    # Ensure inputs are on the same device as the model
    inputs = input_ids.to(next(model.parameters()).device)
    start_time = time.time()
    # Generate outputs from the model
    outputs = model.generate(inputs, max_new_tokens=100, use_cache=True) # can set temperature 
    end_time = time.time()
    # Batch decoding
    text = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    print(text)

    num_tokens = outputs.shape[-1]  # Number of tokens in the output
    time_taken = end_time - start_time
    tokens_per_second = num_tokens / time_taken

    last_eot = text.rfind("<|eot_id|>")

    # Find the second-to-last `[eot]` token (search up to the last one)
    second_last_eot = text.rfind("<|end_header_id|>")

    # Extract the content between them
    if second_last_eot != -1 and last_eot != -1:
        content_between = text[second_last_eot + len("<|end_header_id|>"):last_eot].strip()

        # THIS IS WHERE THE OUTPUT IS PRINTED TO THE TERMINAL
        print(content_between)
    else:
        print("Not enough [eot] tokens found.")
    print(f"----------\nTokens per second: {tokens_per_second}")
    final_time = time.time()
    difference = final_time - end_time
    print(f"\nPost Processing Time: {difference}\n")

hello there! as you may know, i'm passionate about helping people achieve their goals and live a happy and healthy lifestyle. my expertise includes nutrition, fitness, and mindfulness, and i'm here to support you every step of the way.

### Step-by-Step Guide to Achieving Your Goals

#### Step 1: Define Your Goals
- **Reflect on what you want to achieve.** Is it to lose weight, improve your health, or increase your energy levels?
- **Make your goals
Not enough [eot] tokens found.
----------
Tokens per second: 1.0445257847068814

Post Processing Time: 0.026718616485595703

hi, I've been trying to get a new job and have been applying to a variety of positions. However, I've been getting a lot of rejection letters. It's been really frustrating, and I'm starting to feel discouraged. Here are some steps you can take to help you stay motivated and keep your job search on track:

1. **Reflect on Your Experience**: Take some time to review your past work experiences and skills. This can help y